In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/Datasets/Others

/content/drive/My Drive/Datasets/Others


In [3]:
ls

CatVsDog/  Churn_Modelling.csv  data/  Mnist/


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [0]:
#loading Dataset
train_dataset = dsets.MNIST(root = '/Mnist', train= True, download = True, transform= transforms.ToTensor() )

In [0]:
test_dataset = dsets.MNIST(root = '/Mnist', train= False, transform= transforms.ToTensor() )

In [0]:
# making dataset iterable 
# batch size defines here the number of sample that's going to be pass once
# n_iter is defined in such a way to that it manually defined the epoch number

batch_size = 100    
n_iters = 3000
epoch = n_iters / (len(train_dataset)/ batch_size)    # here  it will defined for 5 epochs == 3000 / (60,000 / 100)
epoch = int(epoch)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle = True)
test_load = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [0]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
        
        # Linear function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.relu3 = nn.ReLU()
        
        # Linear function 4 (readout): 100 --> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)
        
        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)
        
        # Linear function 4 (readout)
        out = self.fc4(out)
        return out


In [0]:
# hidden dim == # of neurons
# input dim = size*size of the image
# output dim == # of classes which is 10 in this case

input_dim = 28*28
output_dim = 10
hidden_dim = 100

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [0]:
# initiate loss class

criterion = nn.CrossEntropyLoss()

In [0]:
# backpropagation stuffs

learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [12]:
# training process

iter = 0

for nb_epoch in range(epoch):
    for i,(images,labels) in enumerate(train_loader):
        #load images as variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)

        #clear gradients w.r.t parameters
        optimizer.zero_grad()

        #forward pass to get output 
        output = model(images)
        
        #calculate loss = softmax ->  cross_entropy
        loss = criterion(output, labels)

        #getting gradients w.r.t parameters
        loss.backward()

        #updating parameters 
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            #calculate accuracy
            correct = 0
            total = 0

            #iterate through test dataset 
            for images,labels in test_load:
                images = Variable(images.view(-1,28*28))
                
                output = model(images)

                #get prediction for the output
                _,predicted = torch.max(output.data, 1)

                total += labels.size(0)

                #total correct prediction 
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total 

            #print loss
            print("Iteration: {} | Loss: {} | Accuracy: {}".format( iter ,loss.data,accuracy))            





Iteration: 500 | Loss: 0.2477715164422989 | Accuracy: 90
Iteration: 1000 | Loss: 0.2593822181224823 | Accuracy: 93
Iteration: 1500 | Loss: 0.21495206654071808 | Accuracy: 95
Iteration: 2000 | Loss: 0.11682619154453278 | Accuracy: 95
Iteration: 2500 | Loss: 0.14119859039783478 | Accuracy: 96
Iteration: 3000 | Loss: 0.12458039075136185 | Accuracy: 96
